In [1]:
from inventree.api import InvenTreeAPI
from inventree import company  # noqa: E402
from inventree.stock import StockItem, StockLocation  # noqa: E402
from inventree.company import SupplierPart
from inventree.part import Part, PartAttachment, PartCategory, PartCategoryParameterTemplate, Parameter, ParameterTemplate, PartTestTemplate, PartRelated, BomItem  # noqa:E402

In [2]:
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup
import sys
import getpass
from tqdm import tqdm

In [3]:
#登录Inventree
def login_inventree(url,username,password):
    try:
        api = InvenTreeAPI(host=url, username=username, password=password)
        print('登录成功！')
        return api
    except:
        #print('登录失败！')
        raise(ValueError('登录失败'))

In [4]:
#获取立创商城的订单（手动）保存到txt中
#手动获取订单信息的方法
def read_lcsc_order(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            soup= BeautifulSoup(f.read(), 'html.parser')
            print('文件读取成功！')
            return soup
    except:
        #print('文件读取失败.....')
        raise(ValueError('文件读取失败'))

In [5]:
#处理订单数据
#获取元器件名称 编号 链接 封装 etc
#获取元器件图片链接
def deal_order_data(soup):
    df=pd.DataFrame(columns=['IPN','name','descripition','keyword','footprint','link','img','stock','price'])
    try:
        divs=soup.find_all(class_='order-a-item')
        for i in divs:
            wpercent=i.find_all(class_='w-percent-15')
            dict = {'IPN':i.img['alt'],
                   'link':i.a['href'],
                    'img':i.img['src'],
                    'name':i.li.text.rstrip().split('/')[0].rstrip(),
                    'descripition':i.li.text.rstrip().split('/')[1].lstrip(),
                    'footprint':i.a['title'].split('/')[1].lstrip(),
                    'keyword':i.a.text.strip(),
                    'stock':wpercent[0].text[:-1],
                    'price':wpercent[2].text[1:-2]
                   }
            data = pd.DataFrame(dict,index=[divs.index(i)])
            df=pd.concat([df,data])
            #print(i.img['alt'],i.img['src'],i.img['title'],i.li.text.rstrip(),i.a['href'])
            #print(i.a['title'],i.a.text.strip(),wpercent[0].text[:-1],wpercent[2].text[1:-2])
            download_lcsc_pics(i.img['src'],r'lc/'+i.img['alt']+'.jpg')
        return df
    except:
        print('文件解析失败....')

In [6]:
#下载图片
#便于LCSC的图片上传到Inventree中
#图片命名唯一和IPN一致
def download_lcsc_pics(img_link,img_path):
    urllib.request.urlretrieve(img_link,img_path)

In [7]:
#保存数据到excel中（可选）
def save_datas_to_excel():
    return 0

In [8]:
def Inventree_read_PartCategory(api):
    cats = PartCategory.list(api)
    print('pk ','name')
    for i in cats:
        print(i.pk,'  ',i.name);
    cat_pk=input('请输入插入的分类pk号：')
    return cat_pk

In [9]:
def Inventree_read_company(api):
    coms=company.Company.list(api)
    print('pk ','name')
    for i in coms:
        print(i.pk,'  ',i.name);
    company_pk=input('请输入插入的供应商pk号：')
    return company_pk

In [10]:
def Inventree_read_location(api):
    locs = StockLocation.list(api)
    print('pk ','name')
    for i in locs:
        print(i.pk,'  ',i.name);
    loc_pk=input('请输入插入的默认仓库pk号：')
    return loc_pk

In [11]:
def Inventree_Part_create(api,df,loc_pk,company_pk,cat_pk):
    pbar = tqdm(len(df.index))
    for i in range(0,len(df.index)):
        try:
            couch = Part.create(api, {
                'name': df.loc[i]['name'],
                'description': df.loc[i]['descripition'],
                'category': cat_pk,
                'active': True,
                'virtual': False,
                'IPN':df.loc[i]['IPN'],
                'link':df.loc[i]['link'],
                'keywords':df.loc[i]['keyword'],
                'default_location': loc_pk
                ## Note - You do not have to fill out *all* fields
                })
            couch.uploadImage('lc/'+str(df.loc[i]['IPN'])+'.jpg')
            StockItem.create(api, { 'part': couch.pk, 'quantity': df.loc[i]['stock'], 'location': loc_pk, 'status':10 })
            SupplierPart.create(api,{
            'part':couch.pk,
            'supplier':company_pk,
            'SKU':df.loc[i]['IPN']
            })
            print('导入'+str(df.loc[i]['IPN'])+'成功')
        except:
            print('导入'+str(df.loc[i]['IPN'])+'失败')
        pbar.update(1/len(df.index))
    print('导入完毕')
    pbar.close()

In [12]:
url=input('url:')
username=input('username:')
password=getpass.getpass('password:')
api=login_inventree(url,username,password)

KeyboardInterrupt: Interrupted by user

In [ ]:
filepath=input('file:')
soup=read_lcsc_order(filepath)
df=deal_order_data(soup)
df

In [ ]:
cat_pk=Inventree_read_PartCategory(api)
company_pk=Inventree_read_company(api)
loc_pk=Inventree_read_location(api)

In [ ]:
Inventree_Part_create(api,df,loc_pk,company_pk,cat_pk)